In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import robust_scale
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV



Using TensorFlow backend.
/home/balha/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
#préparation des données pour l'entrainement 
df=pd.read_csv('donee/data.csv',header=0,index_col=0)
X= df.iloc[:,4:-1]
names = X.columns
X=X.values
Y= np.random.choice(2, X.shape[0]).reshape(X.shape[0],1)
for i in range(len(Y)):
    if Y[i]==0:
        X[i,:]=-X[i,:]
X=robust_scale(X)
scaled_df = pd.DataFrame(X, columns=names)
scaled_df.describe()

,DF_ACE,AVR_DF,DF_PFS,DF_PSS,WIN,Break,H2H,DF_POINT,DF_RANK,PPM,complet
count,71679.000000,71679.000000,71679.000000,71679.000000,71679.000000,71679.000000,71679.000000,71679.000000,71679.000000,71679.000000,71679.000000
mean,-0.000638,0.000644,-0.004332,-0.001404,0.001556,0.006915,-0.000014,-0.036516,0.007322,0.001023,-0.006191
std,0.920618,0.865263,1.074684,1.313821,1.309361,1.478742,0.003735,8.870797,1.350745,1.009814,1.179706
min,-6.082584,-9.358946,-17.209744,-11.512323,-5.987196,-12.172979,-1.000000,-96.574468,-4.502513,-33.732877,-6.587534
25%,-0.502183,-0.497278,-0.501302,-0.497837,-0.501067,-0.496321,0.000000,-0.503546,-0.492462,-0.496294,-0.505381
50%,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.497817,0.502722,0.498698,0.502163,0.498933,0.503679,0.000000,0.496454,0.507538,0.503706,0.494619
max,5.534769,8.476495,8.515336,13.367582,5.987196,12.173730,-0.000000,96.560284,4.472362,9.713309,6.917152


In [22]:
# préparation des données pour le test 
ts=pd.read_csv('donnee/ts.csv',header=0,index_col=0)
Xf= ts.iloc[:,5:-1]
Xf = Xf.fillna(Xf.mean(axis=0))
names = Xf.columns
Xf_value=Xf.values
Xf_standard=robust_scale(Xf_value)
scaled_df = pd.DataFrame(Xf_standard, columns=names)
scaled_df.describe()

,DF_ACE,AVR_DF,DF_PFS,DF_PSS,WIN,Break,H2H,DF_POINT,DF_RANK,PPM,complet
count,6097.000000,6097.000000,6097.000000,6097.000000,6097.000000,6097.000000,6097.0,6097.000000,6097.000000,6097.000000,6097.000000
mean,0.010815,-0.039616,-0.081277,-0.130878,0.133199,0.052553,0.0,1.670362,-0.135831,0.046060,0.140895
std,0.932501,0.848859,0.814299,0.849157,0.830071,0.977501,0.0,15.783596,0.987668,0.801167,1.698245
min,-5.626195,-5.327047,-3.196594,-3.733593,-3.179486,-9.165290,0.0,-135.491803,-2.917241,-2.654672,-10.687832
25%,-0.494306,-0.516957,-0.543374,-0.583301,-0.412656,-0.474645,0.0,-0.229508,-0.713793,-0.474900,-0.399440
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,0.505694,0.483043,0.456626,0.416699,0.587344,0.525355,0.0,0.770492,0.286207,0.525100,0.600560
max,5.782259,3.460827,3.098004,3.188654,3.711339,5.628869,0.0,223.032787,2.968966,4.093474,11.090518


In [4]:
features_number=X.shape[1]


In [31]:
# fonction du modèle 
def basic_model(drop_rate,optimizer,init) : 
    model = Sequential()
    model.add(Dense(10, input_dim=features_number,kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dropout(rate=drop_rate))
    model.add(Dense(6,kernel_initializer=init, activation='relu'))
    model.add(Dropout(rate=drop_rate))
    model.add(Dense(4,kernel_initializer=init, activation='relu'))
    model.add(Dropout(rate=drop_rate))
    model.add(Dense(2,kernel_initializer=init, activation='relu'))
    model.add(Dense(1,kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [20]:
# séparation des données 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=0,test_size=0.25)


In [18]:
cfm = confusion_matrix(Y_test, y_pred)

In [19]:
cfm

array([[5985, 2996],
       [2260, 6679]])

In [54]:
# les hyper-paramètres 
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
drop_rate = [0.0, 0.1, 0.2]
epochs = [50, 100, 120]
batches = [5, 10, 20]
# le modèle 
classifier= KerasClassifier(build_fn=basic_model)

In [ ]:
# optimisation des hyper-paramètres par la recherche grid 
param_grid = dict( epochs=epochs, batch_size=batches,drop_rate=drop_rate,optimizer=optimizers,init=init)
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy', cv=2)
grid_result = grid.fit(X_train, Y_train)

/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="glorot_uniform", input_dim=11, activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.0)`
  after removing the cwd from sys.path.
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, kernel_initializer="glorot_uniform", activation="relu")`
  """
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.0)`
  
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, kernel_initializ

Epoch 1/50
26879/26879 [==============================] - 11s 403us/step - loss: 0.5788 - acc: 0.6930
Epoch 2/50
26879/26879 [==============================] - 11s 402us/step - loss: 0.5517 - acc: 0.7073
Epoch 3/50
26879/26879 [==============================] - 10s 379us/step - loss: 0.5492 - acc: 0.7053
Epoch 4/50
26879/26879 [==============================] - 10s 359us/step - loss: 0.5484 - acc: 0.7061
Epoch 5/50
26879/26879 [==============================] - 11s 408us/step - loss: 0.5470 - acc: 0.7083
Epoch 6/50
26879/26879 [==============================] - 10s 389us/step - loss: 0.5466 - acc: 0.7062
Epoch 7/50
26879/26879 [==============================] - 11s 412us/step - loss: 0.5462 - acc: 0.7068
Epoch 8/50
26879/26879 [==============================] - 11s 407us/step - loss: 0.5456 - acc: 0.7075
Epoch 9/50
26879/26879 [==============================] - 11s 412us/step - loss: 0.5454 - acc: 0.7082
Epoch 10/50
26879/26879 [==============================] - 11s 409us/step - loss: 

26880/26880 [==============================] - 9s 348us/step - loss: 0.5461 - acc: 0.7069
Epoch 11/50
26880/26880 [==============================] - 10s 354us/step - loss: 0.5453 - acc: 0.7055
Epoch 12/50
26880/26880 [==============================] - 9s 345us/step - loss: 0.5455 - acc: 0.7082
Epoch 13/50
26880/26880 [==============================] - 9s 349us/step - loss: 0.5454 - acc: 0.7065
Epoch 14/50
26880/26880 [==============================] - 9s 350us/step - loss: 0.5453 - acc: 0.7073
Epoch 15/50
26880/26880 [==============================] - 10s 356us/step - loss: 0.5450 - acc: 0.7075
Epoch 16/50
26880/26880 [==============================] - 9s 351us/step - loss: 0.5449 - acc: 0.7064
Epoch 17/50
26880/26880 [==============================] - 10s 357us/step - loss: 0.5446 - acc: 0.7057
Epoch 18/50
26880/26880 [==============================] - 9s 349us/step - loss: 0.5444 - acc: 0.7080
Epoch 19/50
26880/26880 [==============================] - 9s 351us/step - loss: 0.5440 - a

/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="normal", input_dim=11, activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, kernel_initializer="normal", activation="relu")`
  """
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, kernel_initializer="normal", activation="relu")`
  import sys
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, kernel_initializer="normal", activation="relu")`
  if __name__ == '__main__':
/home/balha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `

Epoch 1/50
26879/26879 [==============================] - 10s 375us/step - loss: 0.6932 - acc: 0.4956
Epoch 2/50
26879/26879 [==============================] - 10s 363us/step - loss: 0.6932 - acc: 0.4977
Epoch 3/50
26879/26879 [==============================] - 10s 365us/step - loss: 0.6932 - acc: 0.4973
Epoch 4/50
26879/26879 [==============================] - 10s 361us/step - loss: 0.6932 - acc: 0.5019
Epoch 5/50
26879/26879 [==============================] - 10s 355us/step - loss: 0.6932 - acc: 0.4964
Epoch 6/50
26879/26879 [==============================] - 10s 361us/step - loss: 0.6932 - acc: 0.4976
Epoch 7/50
26879/26879 [==============================] - 10s 355us/step - loss: 0.6932 - acc: 0.4964
Epoch 8/50
26879/26879 [==============================] - 10s 363us/step - loss: 0.6932 - acc: 0.4999
Epoch 9/50
26879/26879 [==============================] - 10s 355us/step - loss: 0.6932 - acc: 0.4980
Epoch 10/50
26879/26879 [==============================] - 10s 368us/step - loss: 

...

26879/26879 [==============================] - 24s 906us/step - loss: 0.5626 - acc: 0.6909
Epoch 73/100
26879/26879 [==============================] - 24s 902us/step - loss: 0.5653 - acc: 0.6888
Epoch 74/100
26879/26879 [==============================] - 24s 895us/step - loss: 0.5625 - acc: 0.6910
Epoch 75/100
26879/26879 [==============================] - 24s 910us/step - loss: 0.5632 - acc: 0.6917
Epoch 76/100
26879/26879 [==============================] - 24s 904us/step - loss: 0.5624 - acc: 0.6928
Epoch 77/100
26879/26879 [==============================] - 24s 911us/step - loss: 0.5628 - acc: 0.6914
Epoch 78/100
26879/26879 [==============================] - 24s 904us/step - loss: 0.5613 - acc: 0.6932
Epoch 79/100
26879/26879 [==============================] - 24s 900us/step - loss: 0.5635 - acc: 0.6929
Epoch 80/100
26879/26879 [==============================] - 24s 909us/step - loss: 0.5633 - acc: 0.6893
Epoch 81/100
26879/26879 [==============================] - 24s 911us/step - 

Epoch 51/100
26880/26880 [==============================] - 25s 917us/step - loss: 0.5695 - acc: 0.6942
Epoch 52/100
26880/26880 [==============================] - 25s 916us/step - loss: 0.5717 - acc: 0.6914
Epoch 53/100
26880/26880 [==============================] - 25s 920us/step - loss: 0.5711 - acc: 0.6918
Epoch 54/100
26880/26880 [==============================] - 25s 917us/step - loss: 0.5697 - acc: 0.6943
Epoch 55/100
26880/26880 [==============================] - 25s 916us/step - loss: 0.5714 - acc: 0.6955
Epoch 56/100
26880/26880 [==============================] - 24s 911us/step - loss: 0.5717 - acc: 0.6938
Epoch 57/100
26880/26880 [==============================] - 25s 921us/step - loss: 0.5720 - acc: 0.6915
Epoch 58/100
26880/26880 [==============================] - 25s 925us/step - loss: 0.5701 - acc: 0.6959
Epoch 59/100
26880/26880 [==============================] - 25s 928us/step - loss: 0.5681 - acc: 0.6918
Epoch 60/100
26880/26880 [==============================] - 25s 

Epoch 8/100
26880/26880 [==============================] - 25s 916us/step - loss: 0.5733 - acc: 0.6961
Epoch 9/100
26880/26880 [==============================] - 25s 941us/step - loss: 0.5732 - acc: 0.6915
Epoch 10/100
26880/26880 [==============================] - 25s 932us/step - loss: 0.5736 - acc: 0.6941
Epoch 11/100
26880/26880 [==============================] - 25s 936us/step - loss: 0.5736 - acc: 0.6908
Epoch 12/100
26880/26880 [==============================] - 25s 939us/step - loss: 0.5729 - acc: 0.6922
Epoch 13/100
26880/26880 [==============================] - 25s 933us/step - loss: 0.5719 - acc: 0.6927
Epoch 14/100
26880/26880 [==============================] - 25s 942us/step - loss: 0.5732 - acc: 0.6889
Epoch 15/100
26880/26880 [==============================] - 25s 923us/step - loss: 0.5724 - acc: 0.6930
Epoch 16/100
26880/26880 [==============================] - 25s 932us/step - loss: 0.5717 - acc: 0.6920
Epoch 17/100
26880/26880 [==============================] - 27s 99

/home/balha/.local/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.321506). Check your callbacks.
  % delta_t_median)
/home/balha/.local/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.161116). Check your callbacks.
  % delta_t_median)


22775/26879 [========================>.....] - ETA: 4s - loss: 0.5676 - acc: 0.6953

In [ ]:
best_hyperameteres=grid.best_params_
score=grid.best_score_

In [71]:
# le modèle 'optimal' 
model=basic_model(drop_rate=0.1, optimizer='rmsprop',init='glorot_uniform')

In [74]:
# fitting 
model.fit(x=X_train,y=Y_train,batch_size=5,epochs=120)

Epoch 1/120
53759/53759 [==============================] - 31s 578us/step - loss: 0.5589 - acc: 0.6993
Epoch 2/120
53759/53759 [==============================] - 33s 620us/step - loss: 0.5585 - acc: 0.7006
Epoch 3/120
53759/53759 [==============================] - 34s 628us/step - loss: 0.5578 - acc: 0.7018
Epoch 4/120
53759/53759 [==============================] - 30s 552us/step - loss: 0.5569 - acc: 0.7022
Epoch 5/120
53759/53759 [==============================] - 30s 553us/step - loss: 0.5566 - acc: 0.7014
Epoch 6/120
53759/53759 [==============================] - 29s 540us/step - loss: 0.5569 - acc: 0.7026
Epoch 7/120
53759/53759 [==============================] - 30s 554us/step - loss: 0.5563 - acc: 0.7005
Epoch 8/120
53759/53759 [==============================] - 30s 553us/step - loss: 0.5567 - acc: 0.7017
Epoch 9/120
53759/53759 [==============================] - 30s 558us/step - loss: 0.5555 - acc: 0.7016
Epoch 10/120
53759/53759 [==============================] - 30s 559us/ste

In [75]:
# évaluation du modèle 
model.evaluate(x=X_test,y=Y_test,batch_size=10)

17920/17920 [==============================] - 11s 618us/step


[0.55077846877143855, 0.70814732283387072]

In [76]:
# prédiction  de probabilités
y_pred=model.predict(Xf_standard,batch_size=5)

In [81]:
# prédiction du joueur gagneant 
l=[]
for i,j,k in zip(ts['ID1_G'], ts['ID2_G'],list(y_pred)):
    if k > 0.5 :
        l.append(i)
    else:
        l.append(j)

In [83]:
ts['resltat']=l

In [84]:
ts.head(n=10)

,ID1_G,ID2_G,ID_T_G,ID_R_G,DATE_G,DF_ACE,AVR_DF,DF_PFS,DF_PSS,WIN,Break,H2H,DF_POINT,DF_RANK,PPM,complet,FATIGUE,resltat
0,19942,32203,13204,5,2016-05-25 00:00:00,2.299304,0.752671,-0.018580,-0.034057,0.131900,0.067576,0,5,-13,0.031312,57.322563,0.0,19942
1,14609,20648,12992,5,2016-06-01 00:00:00,1.186307,-0.620280,0.013546,0.007328,0.028014,-0.024592,0,-10,28,-0.017927,8.388107,0.0,20648
2,13576,1718,12993,4,2016-06-01 00:00:00,0.387593,-0.802422,-0.017047,-0.018647,0.068954,0.065340,0,-548,270,0.012717,-140.833977,0.0,1718
3,2904,9842,12993,4,2016-06-01 00:00:00,0.168707,0.813826,0.008709,-0.006899,0.011169,-0.001612,0,66,-135,-0.012824,139.311161,0.0,2904
4,4003,1181,12993,4,2016-06-01 00:00:00,1.447193,2.088202,-0.003703,-0.025201,0.033886,0.027632,0,-42,12,0.000453,374.728588,0.0,4003
5,24127,33310,13226,4,2016-06-01 00:00:00,2.555556,1.111111,-0.061114,-0.104319,0.222222,0.225459,0,0,0,0.061726,10.606987,0.0,24127
6,18282,13683,13226,4,2016-06-01 00:00:00,-0.767023,0.471613,-0.012364,-0.051206,0.012903,0.066765,0,3,7,0.071281,-68.372528,0.0,13683
7,12195,9887,13228,5,2016-06-01 00:00:00,-0.051582,0.795479,-0.042836,-0.061261,0.149186,0.074248,0,-58,408,0.055180,19.549321,0.0,9887
8,26890,22906,13228,5,2016-06-01 00:00:00,-1.776231,-0.059362,0.020600,0.051780,-0.119048,0.058365,0,8,-82,0.016924,-67.215673,0.0,22906
9,14465,24009,13228,5,2016-06-01 00:00:00,0.125311,-0.300000,-0.056273,-0.056146,0.175000,0.072800,0,68,-460,0.074973,105.725218,0.0,14465


In [92]:
# prépartion du dataframe résultat 
resultat=pd.DataFrame()
resultat['ID1_G']=ts['ID1_G']
resultat['ID2_G']=ts['ID2_G']
resultat['ID_T_G']=ts['ID_T_G']
resultat['ID_R_G']=ts['ID_R_G']
resultat['Winner']=ts['resltat']





In [94]:
resultat.head(10)

,ID1_G,ID2_G,ID_T_G,ID_R_G,Winner
0,19942,32203,13204,5,19942
1,14609,20648,12992,5,20648
2,13576,1718,12993,4,1718
3,2904,9842,12993,4,2904
4,4003,1181,12993,4,4003
5,24127,33310,13226,4,24127
6,18282,13683,13226,4,13683
7,12195,9887,13228,5,9887
8,26890,22906,13228,5,22906
9,14465,24009,13228,5,14465


In [101]:
# exportation vers fichier CSV sous la forme demandée 
resultat.to_csv('resultat.csv',sep=',', index=True)